Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [22]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader
from sklearn.model_selection import KFold

# Read data

In [7]:
sales = pd.read_csv('/Users/risbhatia/kaggle-practice/predict-future-sales/data/sales_train.csv')

# Aggregate data

In [8]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [16]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [17]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [18]:
all_data.groupby('item_id')['target'].mean()

item_id
0        0.020000
1        0.023810
2        0.019802
3        0.019802
4        0.020000
           ...   
22165    0.021277
22166    0.295918
22167    1.081081
22168    0.032967
22169    0.020833
Name: target, Length: 21807, dtype: float64

In [19]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


#### Method 2

In [20]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [ ]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [21]:
all_data

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042
...,...,...,...,...,...
10768834,59,22162,33,0.0,1.556793
10769024,59,22163,33,0.0,0.581395
10769690,59,22164,33,0.0,1.235589
10771216,59,22166,33,0.0,0.295918


In [36]:
kf = KFold(n_splits=5, shuffle=False)

In [28]:
KFold

/bin/sh: KFold: command not found


In [50]:
rest, curr in kf.split(all_data)

/Users/risbhatia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


(array([      0,       1,       2, ..., 8731077, 8731078, 8731079]), False)

In [51]:
rest_id_mean_map = all_data.iloc[rest].groupby('item_id')['target'].mean()

In [56]:
all_data.iloc[curr,:]['enc'] = all_data.iloc[curr,:]['item_id'].map(rest_id_mean_map)

/Users/risbhatia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
all_data.iloc[curr,]['item_id'].map(rest_id_mean_map)

8513296     0.027304
8513295     0.031646
8510778     2.826790
8508845     0.501295
8512688     0.158983
              ...   
10768834    0.020000
10769024         NaN
10769690         NaN
10771216    0.349227
10770511    1.231425
Name: item_id, Length: 2182770, dtype: float64

In [62]:
enc = all_data.iloc[curr,:]['item_id'].map(rest_id_mean_map)
all_data.iloc[curr]['enc'] = enc

/Users/risbhatia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
enc

8513296     0.027304
8513295     0.031646
8510778     2.826790
8508845     0.501295
8512688     0.158983
              ...   
10768834    0.020000
10769024         NaN
10769690         NaN
10771216    0.349227
10770511    1.231425
Name: item_id, Length: 2182770, dtype: float64

In [65]:
kf.get_n_splits(all_data)
all_data['enc'] = None
for rest, curr in kf.split(all_data):
    rest_id_mean_map = all_data.iloc[rest].groupby('item_id')['target'].mean()
    all_data.loc[curr, 'enc'] = all_data.iloc[curr]['item_id'].map(rest_id_mean_map)
    
    


In [78]:
all_data['enc'].fillna(0.314, inplace=True)

In [77]:
all_data['enc']

139255            NaN
141495      0.0485232
144968       0.142424
142661       0.030303
138947        0.89402
              ...    
10768834         0.02
10769024          NaN
10769690          NaN
10771216     0.349227
10770511      1.23143
Name: enc, Length: 10913850, dtype: object

In [75]:
encoded_feature = all_data['enc'].values

In [76]:
encoded_feature

array([nan, 0.04852320675105485, 0.14242424242424243, ..., nan,
       0.3492268041237113, 1.2314250913520097], dtype=object)

In [79]:
# YOUR CODE GOES HERE
encoded_feature = all_data['enc'].values
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
#grader.submit_tag('KFold_scheme', corr)

0.40921676033777926


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [80]:
del all_data['enc']

In [84]:
all_data['enc'] = \
    (all_data.groupby('item_id')['target'].transform('sum').values - all_data['target'].values)/all_data.groupby('item_id')['target'].transform('count').values

In [86]:
# YOUR CODE GOES HERE
encoded_feature = all_data['enc']
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
#grader.submit_tag('Leave-one-out_scheme', corr)

0.4803928066198171


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [94]:
# smoothing

alpha = 100
g_mean = all_data['target'].mean()

all_data['n_items'] = all_data.groupby('item_id')['target'].transform('count')
all_data['mean_items'] = all_data.groupby('item_id')['target'].transform('mean')

In [97]:
all_data['enc'] = (all_data['mean_items'] * all_data['n_items'] + alpha * g_mean)/ (alpha + all_data['n_items'])

In [99]:
# YOUR CODE GOES HERE

encoded_feature = all_data['enc']
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
#grader.submit_tag('Smoothing_scheme', corr)

0.48181988922695584


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [109]:
# YOUR CODE GOES HERE
all_data['cum_sum'] = all_data.groupby('item_id')['target'].cumsum()
all_data['cum_cnt'] = all_data.groupby('item_id')['target'].cumcount()
encoded_feature = (all_data['cum_sum'] / all_data['cum_cnt']).replace([np.inf, -np.inf], np.nan).fillna(0.33).values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
#grader.submit_tag('Expanding_mean_scheme', corr)

0.5154286003608289


## What did I learn here?

In [108]:
(all_data['cum_sum'] / all_data['cum_cnt']).replace([np.inf, -np.inf], np.nan).fillna(0.33)

139255      0.330000
141495      0.330000
144968      0.330000
142661      0.330000
138947      0.330000
              ...   
10768834    1.560268
10769024    0.585938
10769690    1.238693
10771216    0.296170
10770511    1.081967
Length: 10913850, dtype: float64